## **Systematic study of the effect of the numbers of layers and neurons on the traveltime accuracy**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sciann import Functional, Variable, SciModel
from sciann.utils import *
import scipy.io 
import time
import random

tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

Using TensorFlow backend.


---------------------- SCIANN 0.4.6.2 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 



In [2]:
#Model specifications


v0 = 2.; # Velocity at the origin of the model
vergrad = 0.5; # Vertical gradient
horgrad = 0.; # Horizontal gradient

zmin = 0.; zmax = 2.; deltaz = 0.04;
xmin = 0.; xmax = 2.; deltax = 0.04;


# Point-source location
sz = 1.; sx = 1.;

In [3]:
np.random.seed(123)
tf.random.set_seed(123)

In [4]:
# Creating grid, calculating refrence traveltimes, and prepare list of grid points for training (X_star)

z = np.arange(zmin,zmax+deltaz,deltaz)
nz = z.size

x = np.arange(xmin,xmax+deltax,deltax)
nx = x.size


Z,X = np.meshgrid(z,x,indexing='ij')

# Preparing velocity model
vs = v0 + vergrad*sz + horgrad*sx # Velocity at the source location
velmodel = vs + vergrad*(Z-sz) + horgrad*(X-sx);

# Traveltime solution
if vergrad==0 and horgrad==0: 
  # For homogeneous velocity model
  T_data = np.sqrt((Z-sz)**2 + (X-sx)**2)/v0;
else: 
  # For velocity gradient model
  T_data = np.arccosh(1.0+0.5*(1.0/velmodel)*(1/vs)*(vergrad**2 + horgrad**2)*((X-sx)**2 + (Z-sz)**2))/np.sqrt(vergrad**2 + horgrad**2)


X_star = [Z.reshape(-1,1), X.reshape(-1,1)]

In [5]:
# Analytical solution for the known traveltime part
vel = velmodel[round(sz/deltaz),round(sx/deltax)]

T0 = np.sqrt((Z-sz)**2 + (X-sx)**2)/vel; 

px0 = np.divide(X-sx, T0*vel**2, out=np.zeros_like(T0), where=T0!=0)
pz0 = np.divide(Z-sz, T0*vel**2, out=np.zeros_like(T0), where=T0!=0)

In [6]:
# Find source location id in X_star

TOLX = 1e-6
TOLZ = 1e-6

sids,_ = np.where(np.logical_and(np.abs(X_star[0]-sz)<TOLZ , np.abs(X_star[1]-sx)<TOLX))

print(sids)
print(sids.shape)
print(X_star[0][sids,0])
print(X_star[1][sids,0])

[1300]
(1,)
[1.]
[1.]


In [7]:
layers_list = [2,4,6,8,10]
neurons_list = [2,4,6,8,10]

l2error = np.zeros((len(layers_list),len(neurons_list))) 
rel_l2error = np.zeros((len(layers_list),len(neurons_list))) 

for rcount,num_layers in enumerate(layers_list): 
  for ccount,num_neurons in enumerate(neurons_list): 
    
    print("row=%d, column=%d"%(rcount,ccount))

    tf.random.set_seed(123)  

    # Preparing the Sciann model object

    K.clear_session() 
    layers = [num_neurons]*num_layers

    xt = Variable("xt",dtype='float64')
    zt = Variable("zt",dtype='float64')
    vt = Variable("vt",dtype='float64')
    px0t = Variable("px0t",dtype='float64')
    pz0t = Variable("pz0t",dtype='float64')
    T0t = Variable("T0t",dtype='float64')

    tau = Functional("tau", [zt, xt], layers, 'atan')

    # Loss function based on the factored isotropic eikonal equation
    L = (T0t*diff(tau, xt) + tau*px0t)**2 + (T0t*diff(tau, zt) + tau*pz0t)**2 - 1.0/vt**2

    targets = [tau, L, (1-sign(tau*T0t))*abs(tau*T0t)]
    target_vals = [(sids, np.ones(sids.shape).reshape(-1,1)), 'zeros', 'zeros']

    model = SciModel(
        [zt, xt, vt, pz0t, px0t, T0t], 
        targets
        )

    #Model training

    start_time = time.time()
    hist = model.train(
          X_star + [velmodel,pz0,px0,T0],
          target_vals,
          batch_size = X_star[0].size,
          epochs = 30000,
          learning_rate = 0.0002,
          verbose=0
          )
    elapsed = time.time() - start_time
    print('Training time: %.2f minutes' %(elapsed/60.))

    # Predicting traveltime solution from the trained model

    L_pred = L.eval(model, X_star + [velmodel,pz0,px0,T0])
    tau_pred = tau.eval(model, X_star + [velmodel,pz0,px0,T0])
    tau_pred = tau_pred.reshape(Z.shape)

    T_pred = tau_pred*T0

    print('Time at source: %.4f'%(tau_pred[round(sz/deltaz),round(sx/deltax)]))

    l2error[rcount,ccount] = np.linalg.norm(T_pred-T_data)
    rel_l2error[rcount,ccount] = np.linalg.norm(T_pred-T_data)/np.linalg.norm(T_data)


row=0, column=0

Epoch 16966: ReduceLROnPlateau reducing learning rate to 0.0001.

Epoch 22965: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 28964: ReduceLROnPlateau reducing learning rate to 2.5e-05.
Training time: 2.63 minutes
Time at source: 1.0015
row=0, column=1

Epoch 11680: ReduceLROnPlateau reducing learning rate to 0.0001.

Epoch 20254: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 26253: ReduceLROnPlateau reducing learning rate to 2.5e-05.
Training time: 2.75 minutes
Time at source: 1.0000
row=0, column=2

Epoch 11473: ReduceLROnPlateau reducing learning rate to 0.0001.

Epoch 17472: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 23471: ReduceLROnPlateau reducing learning rate to 2.5e-05.

Epoch 29470: ReduceLROnPlateau reducing learning rate to 1.25e-05.
Training time: 3.20 minutes
Time at source: 1.0000
row=0, column=3

Epoch 17983: ReduceLROnPlateau reducing learning rate to 0.0001.

Epoch 23982: ReduceLROnPlateau reducing learning rate 

Training time: 7.86 minutes
Time at source: 1.0000
row=4, column=4

Epoch 04571: ReduceLROnPlateau reducing learning rate to 0.0001.

Epoch 10570: ReduceLROnPlateau reducing learning rate to 5e-05.

Epoch 16569: ReduceLROnPlateau reducing learning rate to 2.5e-05.

Epoch 22568: ReduceLROnPlateau reducing learning rate to 1.25e-05.

Epoch 28567: ReduceLROnPlateau reducing learning rate to 6.25e-06.
Training time: 9.90 minutes
Time at source: 1.0000


In [8]:
# L2 error
np.set_printoptions(precision=6)
print(l2error)

[[0.166067 0.025429 0.022674 0.013598 0.009014]
 [0.035319 0.024761 0.008673 0.005613 0.007149]
 [0.050591 0.014358 0.014898 0.00861  0.004431]
 [0.013312 0.00819  0.017161 0.013923 0.010156]
 [0.034101 0.005352 0.00667  0.007164 0.00543 ]]


In [9]:
# Relative L2 error
print(rel_l2error)

[[0.009697 0.001485 0.001324 0.000794 0.000526]
 [0.002062 0.001446 0.000506 0.000328 0.000417]
 [0.002954 0.000838 0.00087  0.000503 0.000259]
 [0.000777 0.000478 0.001002 0.000813 0.000593]
 [0.001991 0.000312 0.000389 0.000418 0.000317]]
